Synthesizing SymbTr scores
========

XX

In [ ]:
# import relevant libraries
import json
import os

from symbtrsynthesis.adaptivesynthesizer import AdaptiveSynthesizer


1. Synthesis wrt Arel-Ezgi-Uzdilek theory
------------------------------------

XX

In [ ]:
# define inputs
data_folder = 'sampledata'
score_file = os.path.join(data_folder, 'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.xml')

# synthesize wrt the Arel-Ezgi-Uzdilek theory
AdaptiveSynthesizer.synthesize(score_file, synth_type='karplus', verbose=False)

2.1 Synthesis wrt a reference recording
-----------------------------------

XX

In [ ]:
# reference audio
audio_mbid = 'f970f1e0-0be9-4914-8302-709a0eac088e'
audio_file = os.path.join('sampledata', audio_mbid, audio_mbid + '.mp3')

AdaptiveSynthesizer.synthesize(score_file, reference=audio_file, synth_type='karplus', verbose=False)

2.2 Step by step adaptive synthesis
--------

XX

In [ ]:
# import relevant libraries
import json
import os

from predominantmelodymakam.predominantmelodymakam import PredominantMelodyMakam
from pitchfilter.pitchfilter import PitchFilter
from tonicidentifier.toniclastnote import TonicLastNote
from morty.pitchdistribution import PitchDistribution
from notemodel.notemodel import NoteModel

from symbtrsynthesis.musicxml_reader import read_music_xml
from symbtrsynthesis.adaptivesynthesizer import AdaptiveSynthesizer

from matplotlib import pyplot as plt

%matplotlib inline

2.2.2 Audio Tuning Analysis
----------------------------

In [ ]:
# extract predominant melody
melody_extractor = PredominantMelodyMakam()
pitch = melody_extractor.extract(audio_file)['pitch']

# Post process the pitch track to get rid of spurious pitch estimations and correct octave errors
pitch_filter = PitchFilter()
pitch = pitch_filter.run(pitch)

# tonic identification for the related recording of SymbTr
tonic_identifier = TonicLastNote()
tonic = tonic_identifier.identify(pitch)[0]

# compute the pitch distribution
pitch_distribution = PitchDistribution.from_hz_pitch(pitch[:,1], ref_freq=tonic['value'])

# obtain the tuning
model = NoteModel()
stablenotes = model.calculate_notes(pitch_distribution, tonic['value'], makam,
                                    min_peak_ratio=0.1)

# plot the tuning
model.plot(pitch_distribution, stablenotes)
plt.xlim([160, 450])
plt.show()

2.2.3 Adapting the tuning and synthesizing the SymbTr-score
-----------

This part synthesizes the score according to the related audio recording.
Extracted stable pitches are used as reference in synthesis.

**It takes a time w.r.t. the length of given SymbTr.**
If you want to observe the status of the process, give *"verbose"* flag as *true*.

Possible synth types are *'sine'* and *'karplus'*. *'sine'* option takes less time.

In [ ]:
score = read_music_xml(score_file)
theoretical_histogram, adapted_histogram = AdaptiveSynthesizer.synth_from_tuning(
    score, stable_notes=stablenotes, synth_type='karplus', 
    verbose=False)